# Setup

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import os
import sys
import datetime
import numpy as np
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization, \
                                            Add, Input, Conv1D, MaxPooling1D, Flatten, \
                                            Lambda

from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.metrics import classification_report

In [11]:
os.chdir("/content/drive/MyDrive/Projects/AlarmWaterClassification")

# Utilies

In [12]:
def print_M(conf_M, class_names):
    s = "activity," + ",".join(class_names)
    print(s)
    for i, row in enumerate(conf_M):
        print(class_names[i] + "," + ",".join(map(str, row)))

def print_M_P(conf_M, class_names):
    s = "activity," + ",".join(class_names)
    print(s)
    for i, row in enumerate(conf_M):
        total = sum(row)
        percentages = [str(round(val / total, 2)) if total > 0 else '0' for val in row]
        print(class_names[i] + "," + ",".join(percentages))

def showResult(result, y_test, class_names):
    predictions = [np.argmax(y) for y in result]
    expected = [np.argmax(y) for y in y_test]

    num_labels = y_test.shape[1]
    conf_M = [[0] * num_labels for _ in range(num_labels)]

    for e, p in zip(expected, predictions):
        conf_M[e][p] += 1

    print_M(conf_M, class_names)
    print_M_P(conf_M, class_names)

def load_weight(path):
    model = load_model(path)
    print(model.summary())
    return model

In [13]:
def conv1d_res_block(x, filters, kernel_size, strides=1, activation='relu', padding='valid'):
    x = Conv1D(filters, kernel_size, strides=strides, padding=padding)(x)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    shortcut = x
    x = Conv1D(filters, kernel_size, strides=strides, padding=padding)(x)
    x = BatchNormalization()(x)
    x = Add()([x, shortcut])
    x = Activation(activation)(x)
    return x

def build_improved_model(input_shape, num_labels):
    inputs = Input(shape=(input_shape, 1))

    x = Conv1D(96, 11, strides=4, activation='relu')(inputs)
    x = MaxPooling1D(pool_size=3, strides=2)(x)

    x = conv1d_res_block(x, 256, 5, padding='same')
    x = MaxPooling1D(pool_size=3, strides=2)(x)

    x = conv1d_res_block(x, 384, 3, padding='same')
    x = conv1d_res_block(x, 384, 3, padding='same')
    x = conv1d_res_block(x, 256, 3, padding='same')

    x = MaxPooling1D(pool_size=3, strides=2)(x)

    x = Flatten()(x)

    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_labels, activation='softmax')(x)

    model = tf.keras.models.Model(inputs, outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

# Data

In [14]:
model_weight_out = os.path.join('weights', 'exp_model_16k_1.3.weights.h5')

# if os.path.exists(model_weight_out):
#     sys.exit(f"The same file name exists already: {model_weight_out}")

############### Loading the datasets #####################

print('\nLoading the data\n')

featuresPath = "features_16k/"

class_names = np.load(os.path.join(featuresPath, 'class_names.npy'))

X_train = np.load(os.path.join(featuresPath, 'X_train.npy'))
y_train = np.load(os.path.join(featuresPath, 'y_train.npy'))

X_val = np.load(os.path.join(featuresPath, 'X_val.npy'))
y_val = np.load(os.path.join(featuresPath, 'y_val.npy'))

X_test = np.load(os.path.join(featuresPath, 'X_test.npy'))
y_test = np.load(os.path.join(featuresPath, 'y_test.npy'))

num_labels = y_train.shape[1]


Loading the data



# Data Balancing

In [15]:
print("\nBalancing the data\n")

print("Train Class distribution before balancing:", Counter(np.argmax(y_train, axis=1)))

# Upsampling using SMOTE
smote = SMOTE(sampling_strategy={1: 12000, 2: 10000})
oversampled_features, oversampled_labels = smote.fit_resample(X_train, y_train)

# Downsampling using RandomUnderSampler
undersampler = RandomUnderSampler(sampling_strategy={0: 7300})
undersampled_features, undersampled_labels = undersampler.fit_resample(
    oversampled_features, oversampled_labels)

print("Train Class distribution after balancing:", Counter(
    np.argmax(undersampled_labels, axis=1)))

X_train = undersampled_features
y_train = undersampled_labels


Balancing the data

Train Class distribution before balancing: Counter({0: 7343, 1: 4309, 2: 842})


/usr/local/lib/python3.10/dist-packages/imblearn/utils/_validation.py:313: UserWarning: After over-sampling, the number of samples (12000) in class 1 will be larger than the number of samples in the majority class (class #0 -> 7343)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/imblearn/utils/_validation.py:313: UserWarning: After over-sampling, the number of samples (10000) in class 2 will be larger than the number of samples in the majority class (class #0 -> 7343)
  warnings.warn(


Train Class distribution after balancing: Counter({1: 12000, 2: 10000, 0: 7300})


# Training

In [ ]:
###################### Training the model ###########################3
print("\nTraining the model\n")

model = build_improved_model(X_train.shape[1], num_labels)
# model.summary()

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return float(lr * tf.math.exp(-0.1))

callback = LearningRateScheduler(scheduler)

model.fit(X_train, y_train, batch_size=32, epochs=30,
          validation_data=(X_val, y_val), callbacks=[callback])


Training the model

Epoch 1/30
916/916 [==============================] - 89s 91ms/step - loss: 2.3426 - accuracy: 0.6397 - val_loss: 592.9194 - val_accuracy: 0.5875 - lr: 0.0010
Epoch 2/30
916/916 [==============================] - 82s 90ms/step - loss: 0.6963 - accuracy: 0.6907 - val_loss: 229.4859 - val_accuracy: 0.5875 - lr: 0.0010
Epoch 3/30
916/916 [==============================] - 82s 90ms/step - loss: 0.5816 - accuracy: 0.7594 - val_loss: 227.7922 - val_accuracy: 0.5882 - lr: 0.0010
Epoch 4/30
916/916 [==============================] - 82s 90ms/step - loss: 0.4850 - accuracy: 0.8065 - val_loss: 314.1221 - val_accuracy: 0.5875 - lr: 0.0010
Epoch 5/30
916/916 [==============================] - 82s 89ms/step - loss: 0.4313 - accuracy: 0.8247 - val_loss: 225.6751 - val_accuracy: 0.5875 - lr: 0.0010
Epoch 6/30
916/916 [==============================] - 82s 90ms/step - loss: 0.3701 - accuracy: 0.8501 - val_loss: 148.6884 - val_accuracy: 0.5875 - lr: 0.0010
Epoch 7/30
916/916 [=====

# Testing

In [ ]:
print("\nTesting the model\n")

result = model.predict(X_test)

cnt, cnt_alarm, cnt_other, cnt_water = 0, 0, 0, 0
alarm_num, other_num, water_num = (sum(np.argmax(y_test, axis=1) == 0),
                                    sum(np.argmax(y_test, axis=1) == 1),
                                    sum(np.argmax(y_test, axis=1) == 2))

for i in range(len(y_test)):
    pred = np.argmax(result[i])
    if np.argmax(y_test[i]) == pred:
        cnt += 1
        if pred == 0:
            cnt_alarm += 1
        elif pred == 1:
            cnt_other += 1
        else:
            cnt_water += 1

acc = round(cnt * 100 / len(y_test), 2)
acc_alarm = round(cnt_alarm * 100 / alarm_num, 2)
acc_other = round(cnt_other * 100 / other_num, 2)
acc_water = round(cnt_water * 100 / water_num, 2)

print(
    f"Total Accuracy: {acc}%, Alarm Accuracy: {acc_alarm}%, Others Accuracy: {acc_other}%, Water Accuracy: {acc_water}%")

showResult(result, y_test, class_names)

print("\n")
print(classification_report(
    np.argmax(y_test, axis=1),
    np.argmax(result, axis=1),
    target_names=list(class_names)
))

# Saving

In [ ]:
if not os.path.exists("Models"):
    os.makedirs("Models")
path = os.path.join("Models", f"audio_NN_New{datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}_acc_{acc}")
model_json = model.to_json()
with open(f"{path}.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights(f"{path}.weights.h5")

if not os.path.exists("weights"):
    os.makedirs("weights")
model.save(model_weight_out, overwrite=True, include_optimizer=False)

In [ ]:
loaded_model = tf.keras.models.load_model(model_weight_out)